In [14]:
import pyspark
import pyspark
from pyspark.sql import SparkSession
import os

## DEFINE SENSITIVE VARIABLES
NESSIE_URI = "http://nessie:19120/api/v1"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"



conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
  		#Configuring Catalog
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
        .set('spark.sql.catalog.nessie.s3.endpoint', 'http://minio:9000')
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
  		#MINIO CREDENTIALS
        .set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

## LOAD A CSV INTO AN SQL VIEW
csv_df = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2023.csv")
csv_df.createOrReplaceTempView("csv_open_2023")

## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
spark.sql("CREATE TABLE IF NOT EXISTS nessie.df_open_2023_lesson3 USING iceberg PARTITIONED BY (countryOfOriginCode) AS SELECT * FROM csv_open_2023 ORDER BY countryOfOriginCode;").show()

## ALTER PARTITIONING BASED ON TRUNCATE FIRST LETTER OF firstName
spark.sql("ALTER TABLE nessie.df_open_2023_lesson3 ADD PARTITION FIELD truncate(1, firstName)").show()

## REWRITE ALL THE DATA SO IT ALL USES UPDATED PARTITONING
spark.sql("CALL nessie.system.rewrite_data_files('df_open_2023_lesson3')").show()

Spark Running
++
||
++
++



IllegalArgumentException: Cannot add duplicate partition field null=truncate[1](ref(name="firstName")), conflicts with 1001: firstName_trunc_1: truncate[1](3)

In [15]:
spark.sql("SELECT * FROM nessie.df_open_2023_lesson3").show()

+------------+-------------------+-------------+------------+------+------+-------------------+-------------------+--------+----------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|     competitorName|    firstName|    lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+-------------------+-------------+------------+------+------+-------------------+-------------------+--------+----------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|      455113|      Harmeet Singh|      Harmeet|       Singh|   ACT|     M|                 IN|              India|      28|      Asia|      19739|        CrossFit BFG| 37|180 cm| 78 kg|      16500|       92381|      18|2023|
|     1116202|    Himanshu Mittal|     Himanshu|      Mittal|   ACT|     M|                 IN| 

In [16]:
spark.sql("CREATE VIEW top_ten AS SELECT * FROM nessie.df_open_2023_lesson3 LIMIT 10").show()

AnalysisException: View `default`.`top_ten` already exists. If you want to update the view definition, please use ALTER VIEW AS or CREATE OR REPLACE VIEW AS

In [5]:
spark.sql("SELECT * FROM top_ten").show()

+------------+-----------------+-----------+---------+------+------+-------------------+-------------------+--------+----------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|   competitorName|  firstName| lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+-----------------+-----------+---------+------+------+-------------------+-------------------+--------+----------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|      455113|    Harmeet Singh|    Harmeet|    Singh|   ACT|     M|                 IN|              India|      28|      Asia|      19739|        CrossFit BFG| 37|180 cm| 78 kg|      16500|       92381|      18|2023|
|     1116202|  Himanshu Mittal|   Himanshu|   Mittal|   ACT|     M|                 IN|              India|      28|      A

In [11]:
spark.sql("CREATE TABLE IF NOT EXISTS nessie.top_ten USING iceberg AS SELECT * FROM top_ten ORDER BY lastName").show()

++
||
++
++



In [13]:
spark.sql("SELECT * FROM nessie.top_ten").show()

+------------+-----------------+-----------+---------+------+------+-------------------+-------------------+--------+----------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|   competitorName|  firstName| lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+-----------------+-----------+---------+------+------+-------------------+-------------------+--------+----------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|     2513338|      Hrishi Dave|     Hrishi|     Dave|   ACT|     M|                 IN|              India|      28|      Asia|       7969|CrossFit Life Per...| 31|  null|  null|     117666|      441239|       1|2023|
|     1513429|   Harish Kannuri|     Harish|  Kannuri|   ACT|     M|                 IN|              India|      28|      A